In [19]:
from collections import defaultdict


unit_clause=[]     # list that stores unit clauses


assignment=[]

# method to assign the vlaue to a literal
def assign(literal):
    global assignment
    if literal>0:                         # if the literal is positive then assign true
        assignment[literal]=True
    else:
        assignment[abs(literal)]=False
        
# for the selection of a literal
def jersolow_wang_2_sided_method(sentence):
        score = defaultdict(int)
        for clauses in sentence:
            for literals in clauses:
                score[abs(literals)] += 2 ** -len(clauses)       #calculating the score for each literal 
        return max(score, key=score.get)                    #returning literal with maximum score


######################################################################################################################
'''
  this method have two parts seperated by if and else
  the first part finds the unit literals and update the formula accordingly
  and the second part runs when a literal is passed in flag variable(ie. flag!=0) and the updated formula is returned
  The literal is passed when ther is no unit clause in the formula and a literal is chosen using the above method.


'''
def remove_unit_literals_recursively(sentence,flag):
    global   unit_clause
    
    if flag==0:                            # when the selected literal is not passed
                                            #here we find the unit clauses and return the updated formula 
       
        for ele in sentence:                                # finding unit clauses in current formula
            if len(ele)==1:
                unit_clause.append(ele)
            
        while unit_clause:                  # while there is a unit clause present in the list


            literal=unit_clause[0][0]       #extracting the literal from a unit clause
            
            modified_sentence=[]                 # to store the updated fomula
            assign(literal)
            for ele in sentence:
                if literal in ele:      #not considering  clauses having unit clause's literal with same polarity
                    continue
                if -1*literal in ele:   #considering clauses having unit clause's literal  with opposite polarity
                    temp_clause=[]
                    for ll in ele:
                        if ll!=-1*literal:
                            temp_clause.append(ll)

                    if(len (temp_clause)==0): #when only unit clause's literal with opposite polarity is present-unsatisfiable
                        return 0
                    modified_sentence.append(temp_clause)    
                else:
                    modified_sentence.append(ele)  #when literal in not present in any form:add it to the formula 

            unit_clause=[ele for ele in modified_sentence if len(ele)==1]    #updating the unit clause list 
            sentence=modified_sentence      #updating the formula

            if sentence==0:
                return 0
            if  len(sentence)==0 or sentence==1:
                return 1
        return sentence
            
    else:                #when a literal is chosen and not present in any unit clause
        
        modified_sentence=[]            # to store the updated fomula
        literal=flag                    # the chosen literal is passed in the falg variable
        assign(literal)
        for ele in sentence:
            if literal in ele:     #not considering  clauses having unit clause's literal with same polarity
                continue
            if -1*literal in ele:  #considering clauses having unit clause's literal  with opposite polarity
                temp_clause=[]
                for ll in ele:
                    if ll!=-1*literal:
                        temp_clause.append(ll)

                if(len (temp_clause)==0):  #when only unit clause's literal with opposite polarity is present->unsatisfiable
                    return 0 
                modified_sentence.append(temp_clause)
                
            else:
                modified_sentence.append(ele)   #when literal in not present in any form:add it to the formula

        unit_clause=[ele for ele in modified_sentence if len(ele)==1]    #updating the unit clause list  
        sentence=modified_sentence                         #updating the formula
        
        return sentence          #return the formula

            
def dpll_algorithm(sentence):           
     # base conditions
    if sentence==0:
        return 0
    if sentence==1 or  len(sentence)==0:     
            return 1
     
    flag=0   # flag for the fuction called below to know wheather a literal is passed or not
    sentence=remove_unit_literals_recursively(sentence,flag)   # to remove unit_clauses
   
  
    if sentence==0:                             # when unsatisfiable
        return 0
    if  sentence==1:                        # condition for satisfiability
            return 1
        
    selected_literal = jersolow_wang_2_sided_method(sentence)   # selecting  next literal
    
    # considering both true and false value for seleted variable and returning true if any of them is satisfiable
    return dpll_algorithm(remove_unit_literals_recursively(sentence, selected_literal)) or  dpll_algorithm(remove_unit_literals_recursively(sentence, -1*selected_literal))


if __name__=="__main__":
    filename=input("enter the file name")
 
    sentence=[]  #for storing the formula
   
    global assignment
    
   # Reading the input file and storing the formula in sentence that is a lsit  of lists.
    #Each list of a sentence list is a clause 
    
    with open(filename) as file:         
        for line in file:
            if line[0]=='c':             # skip the comments
                continue
            elif line[0]=='p':                                
                lendata=[int(word) for word in line.split() if word.isdigit()]
                no_of_clauses=lendata[1]                                           #reading number of clauses
                no_of_literals=lendata[0]                                          #reading number of literals
                assignment=[-1 for i in range(no_of_literals+1)]
            else:
                clause=list(map(int, line.split()))               # saving clauses as a list of integers
                lis=[ele for ele in clause if ele!=0] 
                sentence.append(lis)                           #appending clause to the formula
            
    res= dpll_algorithm(sentence)         #apply the dpll algorithum
    
     #print the result in a file output.txt
    index=0
    f = open("output.txt",'w')
    if res==0:                          
        f.write("UNSATISFIABLE")
    else:
        f.write("SATISFIABLE\n")
        for ele in assignment:
            if index!=0:
                f.write(str(index) + " : ")
                f.write(str(ele))
                f.write("\n")
            index+=1
    f.close()
        

enter the file namesat7.cnf
